# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:


# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Task 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
### For this query, the sessionId and itemInSession are composite keys. That way, data will be partitioned by session. Then within each partition, items will be sorted by itemInSession. No record should overwrite each other

In [8]:
# Step1: Create the table 
query = "CREATE TABLE IF NOT EXISTS song_in_session "
query = query + "( sessionId text, itemInSession text, artist text, firstName text, gender varchar,lastName text, length text, level text, location text,   song text, userId text, PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# Step 2: Insert data into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_in_session (sessionId, itemInSession, artist, firstName, gender,  lastName, length, level, location, song, userId)"
        query = query + "VALUES (%s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[8],line[3],line[0], line[1],line[2],line[4],line[5],line[6],line[7],line[9],line[10]))

In [10]:
# Step 3: Select
query = "select artist, song, length from song_in_session WHERE sessionId = '338' AND itemInSession = '4'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length='495.3073')


## Task 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### For this query, we set userid and sessionid as partition key, and iteminsession as clustering key. so that within the userid = 10 sessionid=182 partition, all rows will be sorted by itemInSession


In [11]:
#  Step 1: create the table song_by_user
query = "CREATE TABLE IF NOT EXISTS song_by_user "
query = query + "(userId text, sessionId text, itemInSession text, artist text, firstName text, gender varchar,  lastName text, length text, level text, location text, song text, PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)



In [12]:
# Step 2:  insert data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:        
        query = "INSERT INTO song_by_user (userId, sessionId, itemInSession, artist, firstName, gender,  lastName, length, level, location, song)"
        query = query + "VALUES (%s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[10],line[8],line[3], line[0], line[1],line[2],line[4],line[5],line[6],line[7],line[9]))
                    

In [13]:
#  Step3: SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName from song_by_user WHERE userId = '10' AND sessionId = '182'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)
                    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Task 3:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### This query finds the user by song name
### For this query, we set song as partition key and userId as clustering  key, such that within the song 'All Hands Against His Own', all user records will be sorted by userId, and will not overwrite each other

In [18]:
#Step 1: Create  the table find user_by_song
query = "CREATE TABLE IF NOT EXISTS user_by_song "
query = query + "(song text, userId text, artist text, firstName text, gender varchar, itemInSession text, lastName text, length text, level text, location text, sessionId text,  PRIMARY KEY (song,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
     
                    

In [ ]:
#  step 2:  Insert data into the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_song (song, userId, artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId)"
        query = query + "VALUES (%s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[9],line[10], line[0], line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8]))
 

In [16]:
#  Step 3:  Select data from the table
query = "select firstName, lastName from user_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)  



### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()